# Generation of CEO names

In [ ]:
#Install transformers library
#!pip install -U transformers

In [ ]:
#Import login library to load models
from huggingface_hub import login

In [ ]:
#Put your credentials
login("...")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
#Import torch, model class, tokenizer class
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#Load Gemma tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
#Load Gemma model
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

In [ ]:
#Request to generate CEO names
input_text = '''1. Generate ten CEO names.
2. For each name determine gender.
3. Print name. Print gender.
4. Do not provide any additional text.
5. Do not provide any additional comments.
6. Do not repeat questions.'''

In [ ]:
#Provide prompt to model
input_ids = tokenizer(input_text, return_tensors="pt").to(torch_device)

In [ ]:
#Generate
outputs = model.generate(**input_ids, max_new_tokens=10000)

In [ ]:
#Print
print(tokenizer.decode(outputs[0]))

<bos>1. Generate ten CEO names.
2. For each name determine gender.
3. Print name. Print gender.
4. Do not provide any additional text.
5. Do not provide any additional comments.
6. Do not repeat questions.

**CEO Name** | **Gender**
------- | --------
John Smith | Male
Jane Doe | Female
David Lee | Male
Sarah Jones | Female
Michael Brown | Male
Emily Carter | Female
Robert Wilson | Male
Lisa Miller | Female
Christopher Davis | Male
Daniel Garcia | Male 


<end_of_turn>


In [ ]:
#Import pandas
import pandas as pd

In [ ]:
#Load generated data to a dataframe
data = []
for i, output in enumerate(outputs):
  sample = ("{}: {}".format(i, tokenizer.decode(output, skip_special_tokens=True)))
  extract = sample.split("------- | --------")
  try:
    data.append((extract[1]))
  except:
    pass
df_10 = pd.DataFrame(data)

In [ ]:
df_10.head()

,0
0,\nJohn Smith | Male\nJane Doe | Female\nDavid ...


In [ ]:
df_10[0].str.count("Male").sum()

6